In [1]:
# https://quantumcomputing.stackexchange.com/questions/24050/how-to-implement-a-exponential-of-a-hamiltonian-but-non-unitary-matrix-in-qisk

# control gate: https://qiskit.org/documentation/stubs/qiskit.circuit.ControlledGate.html

# circuit https://www.nature.com/articles/s41598-022-17660-8
import numpy as np

from qiskit.circuit import ControlledGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, Aer, transpile
from qiskit.quantum_info.operators import Operator
from qiskit.extensions import UnitaryGate
from qiskit.tools.visualization import plot_histogram

from qiskit.synthesis import MatrixExponential
from qiskit.quantum_info import Operator
from sklearn.preprocessing import normalize

In [2]:
from qiskit.quantum_info import SparsePauliOp
from qiskit.opflow.list_ops import SummedOp
from qiskit.circuit import Parameter
from qiskit.opflow import I, X, Y, Z, H, CX, Zero, ListOp, PauliExpectation, PauliTrotterEvolution, CircuitSampler, MatrixEvolution, Suzuki, PauliSumOp

shots = 2048
def get_data(noise_mu=0, noise_std=0):
    A = np.array([
    [1.5, 0.5],
    [0.5, 1.5],
    ]).astype('complex')
    
    noise = np.random.normal(noise_mu, noise_std, A.shape)
    A = (A + noise).astype('complex')
    b = np.array([0, 1]).T
    noise = np.random.normal(noise_mu, noise_std, b.shape)
    b = b + noise
    return A, b

In [3]:
def get_gate(A, n):    
    pauli_op = PauliSumOp(SparsePauliOp.from_operator(A))
    phi = Parameter('ϕ')
    evolution_op = (phi * pauli_op).exp_i() # exp(-iϕA)
    trotterized_op = PauliTrotterEvolution(trotter_mode=Suzuki(order=2, reps=1)).convert(evolution_op).bind_parameters({phi: np.pi/n})
    #----control---------
    gate = trotterized_op.to_circuit()
    # print(gate)
    gate.name = f"e^(i*A*pi/{n})"
    gate.label = f"e^(i*A*np.pi/{n})"
    gate = gate.to_gate().control()
    #---------------------
    return gate

In [4]:
def do_circuit(A, n_fourier_modes=3):    
    n_b = 1
    n_ = n_fourier_modes
    n_ancilla = 1
    n_cl = 1
    # quantum circuit initialization
    qc = QuantumCircuit(n_b + n_ + n_ancilla, n_cl)
    # b-vector state preparation
    for i in range(n_b):
        qc.x(i)
    for i in range(n_b, n_b+n_):
        qc.h(i)
    qc.barrier()
    # Matrix exponentiation
    for i in range(0, n_):
        gate = get_gate(A, 2**(i+1))
        qc.append(gate,[i+1, 0])
    qc.barrier()
    # # # Phase estimation
    for j in range(n_b + n_ - 1, n_b, -1):
        qc.h(j)
        for m in range(j - n_b):
            qc.crz(-np.pi/float(2**(j-m - n_b)), j, m+n_b)
    qc.h(j-1)
    qc.barrier()
    # # As I understood, we wncode ancilla qubit to be sure that result will be correct
    for j in range(1, 1+n_):
        qc.cry(np.pi/(2**j), n_b+n_-j, n_b+n_)
    qc.barrier()
    # # Inverse quantum Fourier transform
    for j in range(n_b, n_b + n_):
        for m in range(j - n_b):
            qc.crz(np.pi/float(2**(j-m - n_b)), j, m+n_b)
        qc.h(j)
    qc.h(n_b)
    qc.barrier()
    # # Eigenvalues storing in the vecor b register
    for i in range(n_, 0, -1):
        gate = get_gate(A, -(2**(i)))
        qc.append(gate,[i, 0])
    qc.barrier()
    qc.measure(0, 0)
    # qc.measure(-1, 1)
    # print(qc)
    return qc

In [5]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import depolarizing_error
def get_noise_model():
    # Create an empty noise model
    noise_model = NoiseModel()

    # Add depolarizing error to all single qubit u1, u2, u3 gates
    error = depolarizing_error(0.001, 1)
    noise_model.add_all_qubit_quantum_error(error, ['id', 'u1', 'u2', 'u3', 'rz', 'sx'])
    error = depolarizing_error(0.05, 2)
    noise_model.add_all_qubit_quantum_error(error, ['cx', 'cry', 'crz', 'cp'])
    return noise_model

In [6]:
def simulate(qc, noise):    
    simulator = Aer.get_backend('qasm_simulator')
    circ = transpile(qc, simulator)
    result = execute(qc, backend=simulator, shots=shots, noise_model=noise).result()
    result = execute(qc, backend=simulator, shots=shots).result()
    return result

# Calculated result

In [7]:
def get_probabilities(result):    
    counts = result.get_counts()
    probabilities = counts.copy()
    for k, v in probabilities.items():
        probabilities[k] = v/shots
    return probabilities

# Result vector

In [8]:
def get_vector(probabilities):    
    vect = np.array([v for k,v in probabilities.items()])
    vect.sort()
    return vect

# Real normalized result

In [15]:
def get_real():    
    real_x_norm = [0.25, 0.75] #should be [-0.25, 0.75]
    # real_x_norm = (np.array(a) / np.linalg.norm(a))**2
    real_x_norm.sort()
    return real_x_norm

# Error

In [16]:
def get_mse(vect, real_x_norm):
    mse = (np.square(vect - real_x_norm)).mean()
    return mse

In [19]:
A, b = get_data()
u, s, vh = np.linalg.svd(A, full_matrices=False)
print(f"eigenvalues: {s}")
qc = do_circuit(A, n_fourier_modes=2)
noise=get_noise_model()
result = simulate(qc, noise)
probs = get_probabilities(result)
vect = get_vector(probs)
real_x_norm = get_real()
print(vect, real_x_norm)
mse = get_mse(vect, real_x_norm)
print(mse)

eigenvalues: [2. 1.]
[0.27294922 0.72705078] [0.25, 0.75]
0.0005266666412353516
